# Subclassing gymnasium.Env

This environment implements a very simplistic game consisting of a 2-dimensional square grid of fixed size (specified via the size parameter during construction). The agent can move vertically or horizontally between grid cells in each timestep. The goal of the agent is to navigate to a target on the grid that has been placed randomly at the beginning of the episode.

- States (observations) provide the location of the target and agent.
- There are 4 actions in our environment, corresponding to the movements “right”, “up”, “left”, and “down”.
- A done signal is issued as soon as the agent has navigated to the grid cell where the target is located.
- Rewards are binary and sparse, meaning that the immediate reward is always zero, unless the agent has reached the target, then it is 1.

In [1]:
import numpy as np
import pygame

import gymnasium as gym
from gymnasium import spaces

pygame 2.1.3.dev8 (SDL 2.0.22, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Declaration and Initialization

Our custom environment will inherit from the abstract class `gymnasium.Env`. You shouldn’t forget to add the metadata attribute to your class. There, you should specify the render-modes that are supported by your environment (e.g. `"human"`, `"rgb_array"`, `"ansi"`) and the framerate at which your environment should be rendered. Every environment should support `None` as render-mode; you don’t need to add it in the metadata. In `GridWorldEnv`, we will support the modes “rgb_array” and “human” and render at 4 FPS.

The `__init__` method of our environment will accept the integer `size`, that determines the size of the square grid. We will set up some variables for rendering and define `self.observation_space` and `self.action_space`. In our case, observations should provide information about the location of the agent and target on the 2-dimensional grid. We will choose to represent observations in the form of dictionaries with keys `"agent"` and `"target"`. An observation may look like `{"agent": array([1, 0]), "target": array([0, 3])}`. Since we have 4 actions in our environment (“right”, “up”, “left”, “down”), we will use `Discrete(4)` as an action space. 

In [2]:
class GridWorldEnv(gym.Env):
    metadata = {
        "render_models": ["human", "rgb_array"], 
        "render_fps": 4
    }

    def __init__(self, render_mode=None, size=5):
        self.size = size # size of the square grid
        self.window_size = 512 # size of the PyGame window

        # Observations are dictionaries with the agent's and the target's location.
        # A location is encoded as a tuple (x, y) of ints in {0,...,'size'-1}, i.e. MultiDiscrete([size, size]).

        self.observation_space = spaces.Dict(
            {
                "agent": spaces.Box(0, size - 1, shape=(2, ), dtype=int),
                "target": spaces.Box(0, size - 1, shape=(2, ), dtype=int),
            }
        )

        # We have 4 actions, corresponding to "right", "up", "left", "down"
        self.action_space = spaces.Discrete(4)

        """
        The following dictionary maps abstract actions from 'self.action_space' to the direction we will walk in 
        if that action is taken. I.e. 0 corresponds to "right", 1 to "up" etc.
        """
        self._action_to_direction = {
            0: np.array([1, 0]),
            1: np.array([0, 1]),
            2: np.array([-1, 0]),
            3: np.array([0, -1]),
        }

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode

        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode. They will remain `None` until human-mode is used for the
        first time.
        """
        self.window = None
        self.clock = None

    def _get_obs(self):
        return {"agent": self._agent_location, "target": self._target_location}

    def _get_info(self):
        return {
            "distance": np.linalg.norm(self._agent_location - self._target_location, ord=1)
        }
    
    def reset(self, seed=None, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Choose the agent's location uniformly at random
        self._agent_location = self.np_random.integers(0, self.size, size=2, dtype=int)

        # We will sample the target's location randomly until it does not coincide with the agent's location
        self._target_location = self._agent_location
        while np.array_equal(self._target_location, self._agent_location):
            self._target_location = self.np_random.integers(0, self.size, size=2, dtype=int)
        
        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info

    def step(self, action):
        # Map the action (element of {0, 1, 2, 3}) to the direction we walk in
        direction = self._action_to_direction[action]
        # We use 'np.clip' to make sure we don't leave the grid
        self._agent_location = np.clip(self._agent_location + direction, 0, self.size - 1)
        # An episode is done iff the agent has reached the target
        terminated = np.array_equal(self._agent_location)
        reward = 1 if terminated else 0 # Binary sparse rewards
        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info
        



